# 04. Mô Hình Hóa - Dự Đoán Kết Quả Học Tập
Notebook này thực hiện xây dựng mô hình dự đoán kết quả học tập của sinh viên (Pass/Fail).

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve, average_precision_score
from src.data.loader import DataLoader

# Cấu hình hiển thị
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('Set2')
%matplotlib inline

## 1. Tải và Chuẩn Bị Dữ Liệu

In [ ]:
# Đặt seed để đảm bảo tính tái lập
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

loader = DataLoader()
df = loader.load_combined_data(merge=False)

# Tạo biến mục tiêu (Pass/Fail)
df['pass_fail'] = df['G3'].apply(lambda x: 1 if x >= 10 else 0)  # 1 là Pass, 0 là Fail

print(f"Kích thước dữ liệu: {df.shape}")
print(f"Phân bố lớp mục tiêu:")
print(df['pass_fail'].value_counts())
print(f"Tỷ lệ Pass: {df['pass_fail'].mean()*100:.2f}%")

## 2. Chọn Đặc Trưng và Tạo Tập Dữ Liệu

In [ ]:
# Chọn các đặc trưng cho mô hình
feature_columns = ['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 
                  'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 
                  'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 
                  'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences', 'G1', 'G2']

# Mã hóa các biến phân loại
df_encoded = df.copy()
from sklearn.preprocessing import LabelEncoder

label_encoders = {}
for col in feature_columns:
    if df_encoded[col].dtype == 'object':
        le = LabelEncoder()
        df_encoded[col] = le.fit_transform(df_encoded[col].astype(str))
        label_encoders[col] = le

# Tạo tập đặc trưng và nhãn
X = df_encoded[feature_columns]
y = df_encoded['pass_fail']

print(f"Tập đặc trưng: {X.shape}")
print(f"Tập nhãn: {y.shape}")

## 3. Chia Dữ Liệu

In [ ]:
# Chia dữ liệu thành tập huấn luyện và kiểm thử
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)

print(f"Kích thước tập huấn luyện: {X_train.shape}")
print(f"Kích thước tập kiểm thử: {X_test.shape}")
print(f"Phân bố lớp trong tập huấn luyện:")
print(pd.Series(y_train).value_counts())
print(f"Phân bố lớp trong tập kiểm thử:")
print(pd.Series(y_test).value_counts())

## 4. Chuẩn Hóa Dữ Liệu

In [ ]:
# Chuẩn hóa dữ liệu
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Đã chuẩn hóa dữ liệu")

## 5. Định Nghĩa Các Mô Hình

In [ ]:
# Định nghĩa các mô hình
models = {
    'Logistic Regression': LogisticRegression(random_state=RANDOM_STATE),
    'Random Forest': RandomForestClassifier(random_state=RANDOM_STATE),
    'Gradient Boosting': GradientBoostingClassifier(random_state=RANDOM_STATE),
    'SVM': SVC(random_state=RANDOM_STATE, probability=True)
}

print("Các mô hình đã định nghĩa:")
for name in models.keys():
    print(f"- {name}")

## 6. Huấn Luyện và Đánh Giá Mô Hình với Cross Validation

In [ ]:
# Thực hiện Cross Validation
cv_results = {}
trained_models = {}

# Sử dụng StratifiedKFold để đảm bảo phân bố lớp đồng đều qua các fold
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

for name, model in models.items():
    print(f"\nĐang huấn luyện mô hình: {name}")
    
    # Cross validation
    cv_accuracy = cross_val_score(model, X_train_scaled, y_train, cv=cv, scoring='accuracy')
    cv_precision = cross_val_score(model, X_train_scaled, y_train, cv=cv, scoring='precision')
    cv_recall = cross_val_score(model, X_train_scaled, y_train, cv=cv, scoring='recall')
    cv_f1 = cross_val_score(model, X_train_scaled, y_train, cv=cv, scoring='f1')
    cv_roc_auc = cross_val_score(model, X_train_scaled, y_train, cv=cv, scoring='roc_auc')
    
    # Huấn luyện mô hình trên toàn bộ tập huấn luyện
    model.fit(X_train_scaled, y_train)
    trained_models[name] = model
    
    # Dự đoán trên tập kiểm thử
    y_pred = model.predict(X_test_scaled)
    y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
    
    # Tính các chỉ số trên tập kiểm thử
    test_accuracy = accuracy_score(y_test, y_pred)
    test_precision = precision_score(y_test, y_pred)
    test_recall = recall_score(y_test, y_pred)
    test_f1 = f1_score(y_test, y_pred)
    test_roc_auc = roc_auc_score(y_test, y_pred_proba)
    
    cv_results[name] = {
        'CV_Accuracy_Mean': cv_accuracy.mean(),
        'CV_Accuracy_Std': cv_accuracy.std(),
        'CV_Precision_Mean': cv_precision.mean(),
        'CV_Precision_Std': cv_precision.std(),
        'CV_Recall_Mean': cv_recall.mean(),
        'CV_Recall_Std': cv_recall.std(),
        'CV_F1_Mean': cv_f1.mean(),
        'CV_F1_Std': cv_f1.std(),
        'CV_ROC_AUC_Mean': cv_roc_auc.mean(),
        'CV_ROC_AUC_Std': cv_roc_auc.std(),
        'Test_Accuracy': test_accuracy,
        'Test_Precision': test_precision,
        'Test_Recall': test_recall,
        'Test_F1': test_f1,
        'Test_ROC_AUC': test_roc_auc
    }
    
    print(f"Cross Validation Accuracy: {cv_accuracy.mean():.3f} (+/- {cv_accuracy.std() * 2:.3f})")
    print(f"Cross Validation F1: {cv_f1.mean():.3f} (+/- {cv_f1.std() * 2:.3f})")
    print(f"Test Accuracy: {test_accuracy:.3f}")
    print(f"Test F1: {test_f1:.3f}")

## 7. Bảng So Sánh Mô Hình

In [ ]:
# Tạo bảng so sánh
results_df = pd.DataFrame(cv_results).T
results_df = results_df.round(3)

print("=== BẢNG SO SÁNH KẾT QUẢ MÔ HÌNH ===")
print(results_df[[
    'CV_Accuracy_Mean', 'CV_F1_Mean', 'CV_ROC_AUC_Mean',
    'Test_Accuracy', 'Test_F1', 'Test_ROC_AUC'
]])

## 8. Trực Quan Hóa Kết Quả

In [ ]:
# Trực quan hóa kết quả Cross Validation
metrics = ['CV_Accuracy_Mean', 'CV_F1_Mean', 'CV_ROC_AUC_Mean']
metric_names = ['Accuracy', 'F1-Score', 'ROC-AUC']

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for i, (metric, name) in enumerate(zip(metrics, metric_names)):
    axes[i].bar(results_df.index, results_df[metric], alpha=0.7)
    axes[i].set_title(f'{name} - Cross Validation')
    axes[i].set_ylabel(name)
    axes[i].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Trực quan hóa kết quả Test
test_metrics = ['Test_Accuracy', 'Test_F1', 'Test_ROC_AUC']

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for i, metric in enumerate(test_metrics):
    axes[i].bar(results_df.index, results_df[metric], alpha=0.7, color='orange')
    axes[i].set_title(f'{metric.replace("Test_", "")} - Test Set')
    axes[i].set_ylabel(metric.replace("Test_", ""))
    axes[i].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 9. Phân Tích Chi Tiết Mô Hình Tốt Nhất

In [ ]:
# Xác định mô hình tốt nhất dựa trên F1-score
best_model_name = results_df['Test_F1'].idxmax()
best_model = trained_models[best_model_name]

print(f"Mô hình tốt nhất: {best_model_name}")
print(f"F1-score trên tập kiểm thử: {results_df.loc[best_model_name, 'Test_F1']:.3f}")
print(f"ROC-AUC trên tập kiểm thử: {results_df.loc[best_model_name, 'Test_ROC_AUC']:.3f}")

# Dự đoán với mô hình tốt nhất
y_pred_best = best_model.predict(X_test_scaled)
y_pred_proba_best = best_model.predict_proba(X_test_scaled)[:, 1]

# Báo cáo phân loại chi tiết
print(f"\nBáo cáo phân loại cho mô hình {best_model_name}:")
print(classification_report(y_test, y_pred_best, target_names=['Fail', 'Pass']))

## 10. Ma Trận Nhầm Lẫn

In [ ]:
# Vẽ ma trận nhầm lẫn
plt.figure(figsize=(8, 6))
cm = confusion_matrix(y_test, y_pred_best)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Fail', 'Pass'], yticklabels=['Fail', 'Pass'])
plt.title(f'Ma trận nhầm lẫn - {best_model_name}')
plt.xlabel('Dự đoán')
plt.ylabel('Thực tế')
plt.show()

# Tính toán các chỉ số từ ma trận nhầm lẫn
tn, fp, fn, tp = cm.ravel()
print(f"True Negatives: {tn}")
print(f"False Positives: {fp}")
print(f"False Negatives: {fn}")
print(f"True Positives: {tp}")

## 11. Đường Cong ROC và PR

In [ ]:
# Vẽ đường cong ROC
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
fpr, tpr, _ = roc_curve(y_test, y_pred_proba_best)
auc_score = roc_auc_score(y_test, y_pred_proba_best)
plt.plot(fpr, tpr, label=f'{best_model_name} (AUC = {auc_score:.3f})')
plt.plot([0, 1], [0, 1], 'k--', label='Ngẫu nhiên')
plt.xlabel('Tỷ lệ dương tính giả (FPR)')
plt.ylabel('Tỷ lệ dương tính thật (TPR)')
plt.title('Đường cong ROC')
plt.legend()
plt.grid(True)

# Vẽ đường cong Precision-Recall
plt.subplot(1, 2, 2)
precision_vals, recall_vals, _ = precision_recall_curve(y_test, y_pred_proba_best)
avg_precision = average_precision_score(y_test, y_pred_proba_best)
plt.plot(recall_vals, precision_vals, label=f'{best_model_name} (AP = {avg_precision:.3f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Đường cong Precision-Recall')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

## 12. Điều Chỉnh Siêu Tham Số cho Mô Hình Tốt Nhất

In [ ]:
# Thực hiện tinh chỉnh siêu tham số cho mô hình tốt nhất
if best_model_name == 'Random Forest':
    param_grid = {
        'n_estimators': [100, 200],
        'max_depth': [10, 20, None],
        'min_samples_split': [2, 5],
        'min_samples_leaf': [1, 2]
    }
    base_model = RandomForestClassifier(random_state=RANDOM_STATE)
elif best_model_name == 'Gradient Boosting':
    param_grid = {
        'n_estimators': [100, 200],
        'learning_rate': [0.05, 0.1, 0.2],
        'max_depth': [3, 5, 7]
    }
    base_model = GradientBoostingClassifier(random_state=RANDOM_STATE)
elif best_model_name == 'Logistic Regression':
    param_grid = {
        'C': [0.1, 1, 10],
        'penalty': ['l1', 'l2'],
        'solver': ['liblinear']
    }
    base_model = LogisticRegression(random_state=RANDOM_STATE)
else:  # SVM
    param_grid = {
        'C': [0.1, 1, 10],
        'kernel': ['rbf', 'linear'],
        'gamma': ['scale', 'auto']
    }
    base_model = SVC(random_state=RANDOM_STATE, probability=True)

# Grid search với Cross Validation
grid_search = GridSearchCV(
    base_model, param_grid, cv=cv, scoring='f1', n_jobs=-1, verbose=1
)

print(f"Đang thực hiện tinh chỉnh siêu tham số cho {best_model_name}...")
grid_search.fit(X_train_scaled, y_train)

print(f"Siêu tham số tốt nhất: {grid_search.best_params_}")
print(f"F1-score tốt nhất trong CV: {grid_search.best_score_:.3f}")

# Đánh giá mô hình đã tinh chỉnh trên tập kiểm thử
best_model_tuned = grid_search.best_estimator_
y_pred_tuned = best_model_tuned.predict(X_test_scaled)
y_pred_proba_tuned = best_model_tuned.predict_proba(X_test_scaled)[:, 1]

tuned_accuracy = accuracy_score(y_test, y_pred_tuned)
tuned_f1 = f1_score(y_test, y_pred_tuned)
tuned_roc_auc = roc_auc_score(y_test, y_pred_proba_tuned)

print(f"\nKết quả sau tinh chỉnh siêu tham số:")
print(f"Accuracy: {tuned_accuracy:.3f}")
print(f"F1-score: {tuned_f1:.3f}")
print(f"ROC-AUC: {tuned_roc_auc:.3f}")

## 13. So Sánh Trước và Sau Tinh Chỉnh

In [ ]:
comparison_data = {
    'Metric': ['Accuracy', 'F1-Score', 'ROC-AUC'],
    f'Trước tinh chỉnh ({best_model_name})': [
        results_df.loc[best_model_name, 'Test_Accuracy'],
        results_df.loc[best_model_name, 'Test_F1'],
        results_df.loc[best_model_name, 'Test_ROC_AUC']
    ],
    f'Sau tinh chỉnh ({best_model_name})': [
        tuned_accuracy,
        tuned_f1,
        tuned_roc_auc
    ]
}

comparison_df = pd.DataFrame(comparison_data)
comparison_df = comparison_df.round(3)

print("=== SO SÁNH TRƯỚC VÀ SAU TINH CHỈNH SIÊU THAM SỐ ===")
print(comparison_df)

## 14. Kết Luận

In [ ]:
print("=== KẾT LUẬN MÔ HÌNH HÓA ===")
print(f"\n1. Mô hình tốt nhất: {best_model_name}")
print(f"   - F1-score trên tập kiểm thử: {results_df.loc[best_model_name, 'Test_F1']:.3f}")
print(f"   - ROC-AUC trên tập kiểm thử: {results_df.loc[best_model_name, 'Test_ROC_AUC']:.3f}")

print(f"\n2. Cross Validation:")
print(f"   - Đã sử dụng StratifiedKFold với 5 folds")
print(f"   - Đảm bảo phân bố lớp đồng đều qua các fold")
print(f"   - Random state = {RANDOM_STATE} để đảm bảo tính tái lập")

print(f"\n3. Đánh giá mô hình:")
print(f"   - Đã sử dụng nhiều metric: Accuracy, Precision, Recall, F1, ROC-AUC")
print(f"   - Có phân tích chi tiết qua classification report")
print(f"   - Có trực quan hóa qua confusion matrix, ROC curve, PR curve")

print(f"\n4. Tinh chỉnh siêu tham số:")
print(f"   - Đã thực hiện GridSearchCV với Cross Validation")
print(f"   - So sánh kết quả trước và sau tinh chỉnh")